In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#!pip install rasterio
import rasterio
import xarray as xr

In [19]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

### O3 tif to csv

In [4]:
"""
import rasterio
import pandas as pd
import numpy as np
import os

# Path to the TIFF file
tiff_path = r"D:\satellite Downscaling Mini Project\Satellite-Downscaling-TE-Project\Keigan O3\Maharashtra_O3_July2024-Dec2024.tif"

# Output CSV path
csv_path = r"D:\Satellite Downscaling Mini Project\Satellite-Downscaling-TE-Project\Keigan O3\Maharashtra_O3_July2024-Dec2024.csv"

with rasterio.open(tiff_path) as src:
    data = src.read()  # shape = (bands, rows, cols)

    # Get pixel coordinates
    rows, cols = np.meshgrid(
        np.arange(data.shape[1]),
        np.arange(data.shape[2]),
        indexing='ij'
    )

    # Convert pixel indices to lat/lon
    xs, ys = rasterio.transform.xy(src.transform, rows.flatten(), cols.flatten())
    df = pd.DataFrame({"x": xs, "y": ys})

    # Add each band as a column
    for i in range(data.shape[0]):
        df[f"band_{i+1}"] = data[i].flatten()

# Save DataFrame to CSV
df.to_csv(csv_path, index=False)
print(f"Saved CSV: {csv_path}")
print(df.head())
"""

'\nimport rasterio\nimport pandas as pd\nimport numpy as np\nimport os\n\n# Path to the TIFF file\ntiff_path = r"D:\\satellite Downscaling Mini Project\\Satellite-Downscaling-TE-Project\\Keigan O3\\Maharashtra_O3_July2024-Dec2024.tif"\n\n# Output CSV path\ncsv_path = r"D:\\Satellite Downscaling Mini Project\\Satellite-Downscaling-TE-Project\\Keigan O3\\Maharashtra_O3_July2024-Dec2024.csv"\n\nwith rasterio.open(tiff_path) as src:\n    data = src.read()  # shape = (bands, rows, cols)\n\n    # Get pixel coordinates\n    rows, cols = np.meshgrid(\n        np.arange(data.shape[1]),\n        np.arange(data.shape[2]),\n        indexing=\'ij\'\n    )\n\n    # Convert pixel indices to lat/lon\n    xs, ys = rasterio.transform.xy(src.transform, rows.flatten(), cols.flatten())\n    df = pd.DataFrame({"x": xs, "y": ys})\n\n    # Add each band as a column\n    for i in range(data.shape[0]):\n        df[f"band_{i+1}"] = data[i].flatten()\n\n# Save DataFrame to CSV\ndf.to_csv(csv_path, index=False)\np

### Uv aerosol tif to csv

In [2]:
import rasterio
import pandas as pd
import numpy as np
import os

# Path to the TIFF file
tiff_path = r"D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_UV_Aerosol_July2024-Dec2024.tif"

# Output CSV path
csv_path = r"D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_UV_Aerosol_July2024-Dec2024.csv"

with rasterio.open(tiff_path) as src:
    data = src.read()
    rows, cols = np.meshgrid(                  # Gets pixel coordinates
        np.arange(data.shape[1]),
        np.arange(data.shape[2]),
        indexing='ij'
    )
    xs, ys = rasterio.transform.xy(src.transform, rows.flatten(), cols.flatten())
    df = pd.DataFrame({"x": xs, "y": ys})           #Makes latitude Longitude columns

    #for the actual data value columns
    for i in range(data.shape[0]):
        df[f"band_{i+1}"] = data[i].flatten()
        
df.to_csv(csv_path, index=False)
print(f"Saved CSV: {csv_path}")
print(df.head())

Saved CSV: D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_UV_Aerosol_July2024-Dec2024.csv
           x          y  band_1
0  72.649003  22.028937     NaN
1  72.653494  22.028937     NaN
2  72.657986  22.028937     NaN
3  72.662478  22.028937     NaN
4  72.666969  22.028937     NaN


### Crop Land tif to csv

In [5]:
import rasterio
import pandas as pd
import numpy as np
import os

# Path to the TIFF file
tiff_path = r"D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_Crop_Land_Extent.tif"

# Output CSV path
csv_path = r"D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_Crop_Land_Extent.csv"

with rasterio.open(tiff_path) as src:
    data = src.read()
    rows, cols = np.meshgrid(                  # Gets pixel coordinates
        np.arange(data.shape[1]),
        np.arange(data.shape[2]),
        indexing='ij'
    )
    xs, ys = rasterio.transform.xy(src.transform, rows.flatten(), cols.flatten())
    df = pd.DataFrame({"x": xs, "y": ys})           #Makes latitude Longitude columns

    #for the actual data value columns
    for i in range(data.shape[0]):
        df[f"band_{i+1}"] = data[i].flatten()
        
df.to_csv(csv_path, index=False)
print(f"Saved CSV: {csv_path}")
print(df.head())

Saved CSV: D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_Crop_Land_Extent.csv
           x          y  band_1
0  72.649003  22.028937       0
1  72.653494  22.028937       0
2  72.657986  22.028937       0
3  72.662478  22.028937       0
4  72.666969  22.028937       0


### Evapo Transpiration tif to csv

In [8]:
import rasterio
import pandas as pd
import numpy as np
import os

# Path to the TIFF file
tiff_path = r"D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_Evapo_Transpiration_July2024_Dec2024.tif"

# Output CSV path
csv_path = r"D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_Evapo_Transpiration_July2024_Dec202csv"

with rasterio.open(tiff_path) as src:
    data = src.read()
    rows, cols = np.meshgrid(                  # Gets pixel coordinates
        np.arange(data.shape[1]),
        np.arange(data.shape[2]),
        indexing='ij'
    )
    xs, ys = rasterio.transform.xy(src.transform, rows.flatten(), cols.flatten())
    df = pd.DataFrame({"x": xs, "y": ys})           #Makes latitude Longitude columns

    #for the actual data value columns
    for i in range(data.shape[0]):
        df[f"band_{i+1}"] = data[i].flatten()
        
df.to_csv(csv_path, index=False)
print(f"Saved CSV: {csv_path}")
print(df.head())

Saved CSV: D:\Satellite Downscaling Mini Project\Datasets\Maharashtra_Evapo_Transpiration_July2024_Dec2024.csv
           x          y  band_1
0  72.649003  22.028937     NaN
1  72.653494  22.028937     NaN
2  72.657986  22.028937     NaN
3  72.662478  22.028937     NaN
4  72.666969  22.028937     NaN
